In [18]:
import numpy
from sortedcontainers import SortedDict
import math

In [3]:
timeline = SortedDict()

In [4]:
timeline

SortedDict({})

In [5]:
def ticketgenerator(tau,t, o, a, targ, t2):
    ticket = {
        'tau': tau, #how long the activity lasts
        'time': t, #time at which the activity starts
        'owner': o, #who is doing the activity (could be a female if action=-3)
        'action': a, #which activity (see key)
        'target': targ, #target of the activity (owner=target unless action=-3,-4,-5, or -6)
        'origin': t2 #the time of the ticket that generated this one
    }
    return ticket

In [6]:
# try adding tickets to timeline
def add_to_timeline(ticket):
    global timeline
    timeline[ticket["time"]] = ticket

In [8]:
tmp = ticketgenerator(.1, 0, 5, 2, -3, 1)
add_to_timeline(tmp)
tmp = ticketgenerator(.1, 1.1, 5, 2, -3, 2)
add_to_timeline(tmp)
tmp = ticketgenerator(.1, 0.7, 5, 2, -3, 3)
add_to_timeline(tmp)
tmp = ticketgenerator(.1, 2.0, 5, 2, -3, 4)
add_to_timeline(tmp)
tmp = ticketgenerator(.1, 1.9, 5, 2, -3, 5)
add_to_timeline(tmp)


In [9]:
timeline

SortedDict({0: {'tau': 0.1, 'time': 0, 'owner': 5, 'action': 2, 'target': -3, 'origin': 1}, 0.7: {'tau': 0.1, 'time': 0.7, 'owner': 5, 'action': 2, 'target': -3, 'origin': 3}, 1.1: {'tau': 0.1, 'time': 1.1, 'owner': 5, 'action': 2, 'target': -3, 'origin': 2}, 1.9: {'tau': 0.1, 'time': 1.9, 'owner': 5, 'action': 2, 'target': -3, 'origin': 5}, 2.0: {'tau': 0.1, 'time': 2.0, 'owner': 5, 'action': 2, 'target': -3, 'origin': 4}})

In [10]:
# test building matrix in numpy
nodes = 5
dist_matrix = numpy.zeros(nodes * nodes).reshape((nodes, nodes))

In [11]:
dist_matrix

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [12]:
node_dist = numpy.array([[0.0] * nodes] * nodes)

In [13]:
node_dist

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [14]:
def envgenerator(nodes, x_dim, y_dim):
    Xs = numpy.random.rand(nodes) * x_dim
    Ys = numpy.random.rand(nodes) * y_dim
    return [Xs,Ys]

def dists(xy, bird_speed):
    node_dist = numpy.zeros(nodes * nodes).reshape((nodes, nodes))
    travel_times = numpy.zeros(nodes * nodes).reshape((nodes, nodes))
    for i in range(nodes):
        for j in range(i + 1, nodes):
            dist = math.sqrt((xy[0][j] - xy[0][i]) ** 2 + (xy[1][j] - xy[1][i]) ** 2)
            travel = dist / bird_speed
            node_dist[j][i] = dist
            node_dist[i][j] = dist
            travel_times[j][i] = travel
            travel_times[i][j] = travel
    return [node_dist,travel_times]

xy = envgenerator(5, 4, 5)

In [16]:
xy
nodes = 5

In [19]:
dis = dists(xy, 0.1)

In [24]:
tmp = numpy.exp(-2 * dis[0])

array([[1.00000000e+00, 2.27288951e-04, 1.76011431e-03, 5.30657022e-03,
        1.80585653e-02],
       [2.27288951e-04, 1.00000000e+00, 6.64670619e-03, 2.40930682e-02,
        3.73101122e-03],
       [1.76011431e-03, 6.64670619e-03, 1.00000000e+00, 1.07918795e-01,
        1.76914072e-03],
       [5.30657022e-03, 2.40930682e-02, 1.07918795e-01, 1.00000000e+00,
        1.47848117e-02],
       [1.80585653e-02, 3.73101122e-03, 1.76914072e-03, 1.47848117e-02,
        1.00000000e+00]])

In [34]:
numpy.sum((tmp.transpose() / numpy.sum(tmp, 1)).transpose(), 1)

array([1., 1., 1., 1., 1.])

In [28]:
sum(tmp[0])

1.025352538748554

In [30]:
1 / 1.025352538748554

0.9752743200114403

In [31]:
2.27288951e-04 / 1.025352538748554

0.00022166907713263857

In [39]:
numpy.sum(numpy.arange(9).reshape(3,3), 0)

array([ 9, 12, 15])

In [37]:
numpy.arange(9).reshape(3,3)

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [40]:
improb = 0.99
improb_distance = 800
lamb = -math.log(1 - improb) / improb_distance


In [41]:
lamb

0.005756462732485114

In [49]:
node_dist = dis[0]
visit_preferences = numpy.zeros(nodes * nodes).reshape((nodes, nodes))
for i in range(nodes):
    for j in range(nodes):
        if i!=j:
            visit_preferences[i,j]=math.exp(-lamb*node_dist[i,j])        
    visit_preferences[i]=visit_preferences[i]/sum(visit_preferences[i])

In [50]:
visit_preferences

array([[0.        , 0.24828126, 0.24974832, 0.25054286, 0.25142756],
       [0.24805508, 0.        , 0.25047691, 0.25140705, 0.25006096],
       [0.24901863, 0.24997281, 0.        , 0.25198625, 0.2490223 ],
       [0.24900238, 0.25008907, 0.25117073, 0.        , 0.24973782],
       [0.25073769, 0.24960222, 0.24906672, 0.25059338, 0.        ]])

In [53]:
# compute exponential of each coefficient
visit_preferences = numpy.exp(-lamb * node_dist)
visit_preferences = visit_preferences - numpy.eye(nodes)
# make rows sum to one
visit_preferences = (visit_preferences.transpose() / numpy.sum(visit_preferences, 1)).transpose()

In [54]:
visit_preferences

array([[0.        , 0.24828126, 0.24974832, 0.25054286, 0.25142756],
       [0.24805508, 0.        , 0.25047691, 0.25140705, 0.25006096],
       [0.24901863, 0.24997281, 0.        , 0.25198625, 0.2490223 ],
       [0.24900238, 0.25008907, 0.25117073, 0.        , 0.24973782],
       [0.25073769, 0.24960222, 0.24906672, 0.25059338, 0.        ]])

In [47]:
dis[0,1]

TypeError: list indices must be integers or slices, not tuple

In [48]:
dis

[array([[0.        , 4.19464422, 3.17118826, 2.61940478, 2.00706759],
        [4.19464422, 0.        , 2.50681693, 1.86291555, 2.79553799],
        [3.17118826, 2.50681693, 0.        , 1.11318812, 3.16863066],
        [2.61940478, 1.86291555, 1.11318812, 0.        , 2.10707743],
        [2.00706759, 2.79553799, 3.16863066, 2.10707743, 0.        ]]),
 array([[ 0.        , 41.9464422 , 31.7118826 , 26.19404781, 20.07067588],
        [41.9464422 ,  0.        , 25.06816929, 18.62915554, 27.95537988],
        [31.7118826 , 25.06816929,  0.        , 11.13188117, 31.6863066 ],
        [26.19404781, 18.62915554, 11.13188117,  0.        , 21.07077431],
        [20.07067588, 27.95537988, 31.6863066 , 21.07077431,  0.        ]])]